In [1]:
import numpy as np
import pandas as pd

In [15]:
import scattertext as st

In [4]:
df = pd.read_pickle("./df_all.pkl")

**Try BMI**

In [5]:
# Data Frame already has BMI column that was created in earlier analysis
## I'll drop it and re-create once again

df.drop('BMI', axis=1, inplace=True) 

In [7]:
df['BMI'] = df['height'] + ' ' + df['weight']

In [8]:
def BMI(x):
    if x == np.nan:
        return np.nan
    else:
        try:
            h = (int(x.split(':')[1].split(' ')[1][0])*12 + int(x.split(':')[1].split(' ')[2][0]))**2
            w = int(x.split(':')[-1].strip())
            bmi = round((w/h)*703,2)
        except (IndexError, AttributeError, ValueError):
            bmi = np.nan
        return bmi

In [9]:
df['BMI'] = df['BMI'].apply(BMI)

ScatterText steps

In [13]:
# Separating data into higher BMI and lower BMI
df['body_type'] = np.where(df['BMI'] >= 24.5, 'higher BMI', 'lower BMI')

In [14]:
df.columns

Index(['name', 'review_title', 'review_body', 'rating', 'date_posted',
       'usual_size', 'size_purchased', 'height', 'fit', 'weight',
       'product_type', 'review_clean', 'review_nltk', 'majority',
       'height_type', 'BMI', 'body_type'],
      dtype='object')

In [22]:
stop_words_scatter = (['everlane', 'j', 'crew', 'holy', 'stress', 'fyi', 'name', 'words', 'and/or',
                      'hello', 'the', 'off', 'no', 'a', 'e', 'c', 'tts', 'jcrew', 'vs',
                      'lol', 'jeans', 'darn', 'sz', 'not', 'me', 'ie', 'ca', 'ish', 'aka', 'levi', 'eagle', 'cm',
                      'grail', 'hi', 'us', 'they', 'l', 'sf', 'asap', 'yes', 'which', 'gosh', 'notch', 'fav', 'h',
                      'ny', 'madewell', 'girl', 'ps', 'cheeky', 'skinny', 'has', 'have', 'it', 'uk', 'con',
                      'has', 'own', 'highrise', 'highwaisted', 'none', 'midrise', 'are', 'sister', 'theyre', 'theyll',
                      'white', 'fit', 'light', 'highwaist', 'midblue', 'summer', 'normal', 'crew', 'everlane', 'ive'])

In [23]:
def another_cleaning(text, stop_list):
    list_words = text.split(' ')
    for word in list_words:
        if word in stop_list:
            list_words.remove(word)
    return ' '.join(list_words)

In [24]:
df['review_nltk'] = df['review_nltk'].apply(another_cleaning, stop_list=stop_words_scatter)

In [25]:
corpus = st.CorpusFromPandas(df,
                             category_col='body_type',
                             text_col='review_nltk',
                             nlp=st.whitespace_nlp_with_sentences
                            ).build()

In [26]:
html = st.produce_scattertext_explorer(
        corpus,
        category='higher BMI',
        category_name='higher BMI',
        not_category_name='lower BMI',
        minimum_term_frequency=5,
        pmi_threshold_coefficient=5,
        width_in_pixels=1000,
        metadata=df['name'],
        )

open('BMI.html', 'wb').write(html.encode('utf-8'));